# Python для сбора и анализа данных

*Алла Тамбовцева*

## Практикум 3*. Работа с API ВКонтакте: собираем посты со стены

## Подготовка к работе

Загружаем модули и библиотеки, необходимые для работы:

In [1]:
import requests
import time
import pandas as pd

Для начала давайте посмотрим на документацию API и посмотрим, как к нему формировать запросы: https://dev.vk.com/api/api-requests.
По [инструкции](https://allatambov.github.io/pypolit/vk-auth.pdf) мы можем получить доступ к API, создадим приложение и скопируем его ID, чтобы получить ссылку для токена доступа:

In [2]:
app_id = input("Enter your client id: ")
url = f"https://oauth.vk.com/authorize?client_id={app_id}&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token"
print(url)

Enter your client id: 51852748
https://oauth.vk.com/authorize?client_id=51852748&display=page&redirect_uri=http://oauth.vk.com/blank.html&scope=all&response_type=token


Переходим по ссылке выше и копируем ссылку (полностью!) из адресной строки:

In [3]:
full_link = input()

https://oauth.vk.com/blank.html#access_token=vk1.a.j34Ke1lS2YY3PigYymioyVve6zZOjzhk4QQ9g-IXK7K_ua6w185G-h5Z3RkNc3W9Yyam1VKPVsR39gIgXV5tMSMwNE9uhRU0bixfL-XNTrOYT4B7dbZCr8mbtJs21hQpp_xQ03OpZA28C8SA88DUNYNo61lYUgi3Kh8NYAfKFm9vXd5zmLWa370s2wjEGy81&expires_in=86400&user_id=20473269


Разбиваем полученную ссылку (сначала по `access_token=`, потом часть после `=` по `&`) и забираем токен в чистом виде:

In [4]:
token = full_link.sp§lit("access_token=")[1].split("&")[0]

In [5]:
token

'vk1.a.j34Ke1lS2YY3PigYymioyVve6zZOjzhk4QQ9g-IXK7K_ua6w185G-h5Z3RkNc3W9Yyam1VKPVsR39gIgXV5tMSMwNE9uhRU0bixfL-XNTrOYT4B7dbZCr8mbtJs21hQpp_xQ03OpZA28C8SA88DUNYNo61lYUgi3Kh8NYAfKFm9vXd5zmLWa370s2wjEGy81'

Ура! Теперь токен доступа у нас есть, всё готово к работе!

## Выгружаем посты со стены сообщества

На этом практическом занятии мы будем выгружать посты из сообщества [скалодрома Rock Zona](https://vk.com/rzclimbing). Сохраним в переменные версию API, ссылку для метода работы со стеной сообщества и название сообщества:

In [6]:
v = "5.131"
main_wall = "https://api.vk.com/method/wall.get"
domain = "rzclimbing"

Функция `get()` из библиотеки `requests` умеет подставлять в запрос необходимые параметры и объединять их с помощью `?` и `&`. Сохраним необходимые параметры в виде словаря:

In [7]:
params_wall = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v}

А теперь сформируем запрос и выгрузим результаты в формате JSON – в Python данные в таком формате будут представлены в виде словаря:

In [8]:
req_wall = requests.get(main_wall, params = params_wall)

In [9]:
json_wall = req_wall.json()
#json_wall

In [10]:
json_wall

{'response': {'count': 1750,
  'items': [{'inner_type': 'wall_wallpost',
    'donut': {'is_donut': False},
    'comments': {'can_post': 1, 'count': 2, 'groups_can_post': True},
    'marked_as_ads': 0,
    'short_text_rate': 0.8,
    'hash': 'hQ9zWQBikTFKamuZBnyWipNz20U',
    'type': 'post',
    'carousel_offset': 0,
    'attachments': [{'type': 'photo',
      'photo': {'album_id': -7,
       'date': 1709142054,
       'id': 457244882,
       'owner_id': -38936316,
       'access_key': '3c3110831a8582a2fb',
       'sizes': [{'height': 75,
         'type': 's',
         'width': 75,
         'url': 'https://sun1-55.userapi.com/impg/Szf8I4gsRFar2442-DCmKxxmCSmN8puBKGATpg/3SgiMy3Ezr4.jpg?size=75x75&quality=95&sign=845c75bbf5c5a6607872133d4855dafa&c_uniq_tag=cwHg20KB-Jj1UKdMflb0txYoRErvJoon9bVyEh3_2gc&type=album'},
        {'height': 130,
         'type': 'm',
         'width': 130,
         'url': 'https://sun1-55.userapi.com/impg/Szf8I4gsRFar2442-DCmKxxmCSmN8puBKGATpg/3SgiMy3Ezr4.jpg?size

Извлечём из этого большого словаря элемент, который отвечает за общее число постов на стене:

In [11]:
nposts = json_wall['response']['count']
print(nposts)

1750


Теперь извлечём элемент, который хранит результаты – список из маленьких словарей с информацией о постах (1 словарь = 1 пост):

In [12]:
items_wall = json_wall['response']['items']

Посмотрим на один элемент такого списка:

In [13]:
i = items_wall[0]
i

{'inner_type': 'wall_wallpost',
 'donut': {'is_donut': False},
 'comments': {'can_post': 1, 'count': 2, 'groups_can_post': True},
 'marked_as_ads': 0,
 'short_text_rate': 0.8,
 'hash': 'hQ9zWQBikTFKamuZBnyWipNz20U',
 'type': 'post',
 'carousel_offset': 0,
 'attachments': [{'type': 'photo',
   'photo': {'album_id': -7,
    'date': 1709142054,
    'id': 457244882,
    'owner_id': -38936316,
    'access_key': '3c3110831a8582a2fb',
    'sizes': [{'height': 75,
      'type': 's',
      'width': 75,
      'url': 'https://sun1-55.userapi.com/impg/Szf8I4gsRFar2442-DCmKxxmCSmN8puBKGATpg/3SgiMy3Ezr4.jpg?size=75x75&quality=95&sign=845c75bbf5c5a6607872133d4855dafa&c_uniq_tag=cwHg20KB-Jj1UKdMflb0txYoRErvJoon9bVyEh3_2gc&type=album'},
     {'height': 130,
      'type': 'm',
      'width': 130,
      'url': 'https://sun1-55.userapi.com/impg/Szf8I4gsRFar2442-DCmKxxmCSmN8puBKGATpg/3SgiMy3Ezr4.jpg?size=130x130&quality=95&sign=13765bb5e12b42b19c4a344e1864b480&c_uniq_tag=tVEKDxL4ZOlCpBXhmS5GWXSVvmwZ8vBsl0n

### Задача 1

Извлеките из элемента `i` следующие компоненты:

* id поста;
* дата поста;
* текст поста;
* число лайков;
* число репостов;
* число просмотров;
* число комментариев.

In [19]:
#print(i["id"], i["date"], i["text"])
print(i["comments"]["count"], i["likes"]["count"], i["views"]["count"])

2 20 513


### Задача 2

Изучить один пост и понять, что нам от него нужно, это хорошо, но, конечно, мы захотим выгрузить все посты сразу, а уже потом разобраться, какую информацию о них нам оставить. Ограничения данного API таковы, что за один раз мы можем выгрузить только 100 постов. Хорошие новости: каждый раз при выгрузке мы можем начинать с того поста, на котором закончили, то есть сначала выгрузить первые 100 постов, потом – следующие 100 постов, и так до тех пор, пока не заполучим все. 

Общее число постов сохранено в `nposts`. Посчитайте, сколько раз нужно будет выполнить выгрузку по 100 постов, чтобы собрать все тексты, и сохраните его в переменную `iterate`.

In [20]:
iterate = nposts // 100 + 1

### Задача 3

Прочитайте в документации к API ВКонтакте про аргумент `offset` в методе `wall.get`. Используя полученную информацию и блоки кода ниже, выгрузите и сохраните в список `items_all` данные по всем постам на стене сообщества.

**Подсказка:** чтобы расширять список правильным образом, используйте метод `.extend()`, а не `.append()`, он добавляет не один элемент, а сразу несколько.

In [21]:
params_wall_long = {"access_token" : token, 
              "domain" : domain, 
              "count" : 100,
              "v" : v,
              "offset" : 0}

In [22]:
items_all = []

for i in range(iterate):
    req_wall_long = requests.get(main_wall, params = params_wall_long)
    json_wall_long = req_wall_long.json()
    items_wall_long = json_wall_long['response']['items']
    
    # YOUR CODE HERE
    items_all.extend(items_wall_long)
    params_wall_long["offset"] += 100
    
    time.sleep(1)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17


Проверяем длину списка – все ли посты собраны:

In [ ]:
len(items_all)

### Задача 4

Создайте на основе списка `items_all` датафрейм `df` со следующими столбцами:

* id поста (`id`);
* дата поста (`date`);
* текст поста (`text`);
* число лайков (`nlikes`);
* число просмотров (`nviews`);
* число комментариев (`ncomments`).

**Подсказка:** отберите сначала необходимую информацию из каждого элемента в `items_all`, а затем переходите к датафрейму.

In [ ]:
### YOUR CODE HERE ###